# Pose detection algorithm using PoseNet

In [ ]:
# Checking Python Version

import sys
print("Python version: "+sys.version)

In [ ]:
# Mounting Google drive to use resources from it

from google.colab import drive
drive.mount("/content/drive/")

In [ ]:
# Imports

import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw, ImageColor
import os
import matplotlib.pyplot as plt
import cv2
import pandas as pd

# Keypoint Calculation

In [ ]:
class Pose:
  KEYPOINTS = (
  'nose',
  'left eye',
  'right eye',
  'left ear',
  'right ear',
  'left shoulder',
  'right shoulder',
  'left elbow',
  'right elbow',
  'left wrist',
  'right wrist',
  'left hip',
  'right hip',
  'left knee',
  'right knee',
  'left ankle',
  'right ankle')

  EDGES = (
      ('nose', 'left eye'),
      ('nose', 'right eye'),
      ('nose', 'left ear'),
      ('nose', 'right ear'),
      ('left ear', 'left eye'),
      ('right ear', 'right eye'),
      ('left eye', 'right eye'),
      ('left shoulder', 'right shoulder'),
      ('left shoulder', 'left elbow'),
      ('left shoulder', 'left hip'),
      ('right shoulder', 'right elbow'),
      ('right shoulder', 'right hip'),
      ('left elbow', 'left wrist'),
      ('right elbow', 'right wrist'),
      ('left hip', 'right hip'),
      ('left hip', 'left knee'),
      ('right hip', 'right knee'),
      ('left knee', 'left ankle'),
      ('right knee', 'right ankle'),
  )

  def __init__(self, model_path):
    self.tflite_interpreter = tf.lite.Interpreter(model_path=model_path)
    self.tflite_interpreter.allocate_tensors()

    self.input_details = self.tflite_interpreter.get_input_details()
    self.output_details = self.tflite_interpreter.get_output_details()

  @staticmethod
  def _sigmoid(z):
    return 1/(1 + np.exp(-z))

  # Calculation of Keypoints
  def calc(self, img):
    self.tflite_interpreter.set_tensor(self.input_details[0]['index'], np.expand_dims(np.asarray(img).astype('float32')/ 128.0 - 1.0, axis=0))
    self.tflite_interpreter.invoke()

    output_tensor = [self.tflite_interpreter.get_tensor(self.output_details[i]["index"]) for i in range(len(self.output_details))]

    # 1 * 9 * 9 * 17 contains heatmaps
    heatmapsShape = output_tensor[0].shape

    # 1 * 9 * 9 * 34 contains offsets
    offsetsShape = output_tensor[1].shape

    # 1 * 9 * 9 * 32 contains forward displacements
    displacementsFwdShape = output_tensor[2].shape

    # 1 * 9 * 9 * 32 contains backward displacements
    displacementsBwdShape = output_tensor[3].shape


    heatmaps = np.asarray(output_tensor[0])
    offsets = np.asarray(output_tensor[1])

    height = heatmaps[0].shape[0]
    width = heatmaps[0].shape[1]
    numKeypoints = heatmaps[0][0][0].size

    # Finds the (row, col) locations of where the keypoints are most likely to be.
    keypointPositions = []
    for keypoint in range(numKeypoints):
      maxVal = heatmaps[0][0][0][keypoint]
      maxRow = 0
      maxCol = 0
      for row in range(height):
        for col in range(width):
          if (heatmaps[0][row][col][keypoint] > maxVal):
            maxVal = heatmaps[0][row][col][keypoint]
            maxRow = row
            maxCol = col
      keypointPositions.append((maxRow, maxCol))


    # Calculating the x and y coordinates of the keypoints with offset adjustment.
    output_dic = {}
    total_score = 0.0
    for idx, (bodypart, (positionY,positionX)) in enumerate(zip(Pose.KEYPOINTS, keypointPositions)):
      output_dic[bodypart] = {}
      output_dic[bodypart]['x'] = int(positionX / (width - 1) * img.shape[1] + offsets[0][positionY][positionX][idx + numKeypoints])
      output_dic[bodypart]['y'] = int(positionY / (height - 1) * img.shape[0] + offsets[0][positionY][positionX][idx])
      output_dic[bodypart]['score'] = self._sigmoid(heatmaps[0][positionY][positionX][idx])
      total_score += output_dic[bodypart]['score']

    output_dic['total_score'] = total_score / len(Pose.KEYPOINTS)

    return output_dic

  # Optional method that plots the keypoints and edges on the original image
  def draw_pose(self, pose, img, threshold=0.5, marker_color='green', color='yellow', marker_size=5, thickness=2):
    img1 = Image.fromarray(np.uint8(img)).convert('RGB')
    img1 = img1.resize((257,257))

    draw = ImageDraw.Draw(img1)

    edges = []
    for p1, p2 in Pose.EDGES:
        if (pose[p1]['score'] < threshold) or (pose[p2]['score'] < threshold): continue
        draw.line((pose[p1]['x'], pose[p1]['y'], pose[p2]['x'], pose[p2]['y']), fill=color, width=thickness)
        edges.append((pose[p1]['x'], pose[p1]['y'], pose[p2]['x'], pose[p2]['y']))

    for label, keypoint in pose.items():
      if label=='total_score' : break
      if keypoint['score'] < threshold: continue

      draw.ellipse((int(keypoint['x']-marker_size/2), int(keypoint['y']-marker_size/2), int(keypoint['x']+marker_size/2), int(keypoint['y']+marker_size/2)), fill=marker_color)

    return img1

# Data saving to CSV

In [ ]:
  # Transforms data to be saved in respective CSV file
  def saveData(pose ,code, base):
    path = "/content/drive/MyDrive/Colab Notebooks/"
    keypointData = pd.DataFrame(pose.items())
    keypointData[1] = keypointData[1].apply(lambda x: str(x).replace(',', ';'))

    formatted_data = ", ".join([f"{value}:{keypointData[1][idx]}" for idx, value in enumerate(keypointData[0])])
    formatted_df = pd.DataFrame([formatted_data.split(',')]).T

    flat = formatted_df.T.values.flatten()
    flat_df = pd.DataFrame(flat).T
    flat_df.insert(loc=0, column="id", value=code)
    flat_df.to_csv(path+"keypoints_"+base+".csv", index=False, header=False, mode='a')

# Loading images and creating custom image code

In [ ]:
def image_loader(directory):
    os.chdir(directory + "/High-Kick")
    img_list = os.listdir()
    norm = os.path.normpath(directory)
    base = os.path.basename(norm)
    # iterate through all images in the directory
    for image in img_list:
        # loads the image
        img = cv2.imread(image)
        # gets image name
        name = os.path.splitext(image)[0]
        # extracts custom ID code from image name
        code = name[0:12]
        # removes any trailing underscores
        if code[-1] == "_":
            code = code[:-1]
        # sends image to pose estimation model
        output_dict = pose.calc(img)
        # saves data to CSV
        saveData(output_dict, code, base)
    print(directory + " directory High-Kick images completed")

    os.chdir(directory + "/Knee-Strike")
    img_list = os.listdir()
    norm = os.path.normpath(directory)
    base = os.path.basename(norm)
    # iterate through all images in the directory
    for image in img_list:
        # loads the image
        img = cv2.imread(image)
        # gets image name
        name = os.path.splitext(image)[0]
        # extracts custom ID code from image name
        code = name[0:12]
        # removes any trailing underscores
        if code[-1] == "_":
            code = code[:-1]
        # sends image to pose estimation model
        output_dict = pose.calc(img)
        # saves data to CSV
        saveData(output_dict, code, base)
    print(directory + " directory Knee-Strike images completed")

    os.chdir(directory + "/Low-Kick")
    img_list = os.listdir()
    norm = os.path.normpath(directory)
    base = os.path.basename(norm)
    # iterate through all images in the directory
    for image in img_list:
        # loads the image
        img = cv2.imread(image)
        # gets image name
        name = os.path.splitext(image)[0]
        # extracts custom ID code from image name
        code = name[0:12]
        # removes any trailing underscores
        if code[-1] == "_":
            code = code[:-1]
        # sends image to pose estimation model
        output_dict = pose.calc(img)
        # saves data to CSV
        saveData(output_dict, code, base)
    print(directory + " directory Low-Kick images completed")

In [ ]:
# https://www.tensorflow.org/lite/models/pose_estimation/overview
!wget "https://storage.googleapis.com/download.tensorflow.org/models/tflite/posenet_mobilenet_v1_100_257x257_multi_kpt_stripped.tflite"

# Run

In [ ]:
pose = Pose("posenet_mobilenet_v1_100_257x257_multi_kpt_stripped.tflite")

# setup paths for dataset
data_dir = '/content/drive/MyDrive/V1 Dataset'
val_dir = data_dir + '/valid'
test_dir = data_dir + '/test'
train_dir = data_dir + '/train'

image_loader(val_dir)
image_loader(test_dir)
image_loader(train_dir)

print("finished")